In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1
pip install tensorboardX scipy

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

In [ ]:
from huggingface_hub import notebook_login

import subprocess

subprocess.run("huggingface-cli login", shell=True)

notebook_login()

In [ ]:
# The model that you want to train from the Hugging Face hub
base_model = "google/gemma-7b-it"

# The instruction dataset to use
dataset_name = "CodeTriad/Fine-tuning_dataset_gemma_ing_title

# Fine-tuned model name
new_model = "CodeTriad/gemma_finetune_15000_2"

In [ ]:
# Load dataset (you can process it here)
train_data_files = {"train": "fine-tuning-dataset_gemma_train_15000.csv"}
validation_data_files = {"validation": "fine-tuning-dataset_gemma_val_6000.csv"}

train_dataset = load_dataset(dataset_name, data_files=train_data_files, split="train")
validation_dataset = load_dataset(dataset_name, data_files=validation_data_files, split="validation")

In [ ]:
token =''
quantization_config = BitsAndBytesConfig(load_in_16bit=True)

tokenizer = AutoTokenizer.from_pretrained("google/gemma-7b-it",token = token)
model = AutoModelForCausalLM.from_pretrained("google/gemma-7b-it", quantization_config=quantization_config,token = token)

In [7]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=80,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=500,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    evaluation_strategy="epoch",
    report_to="tensorboard"
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="prompt",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

In [ ]:
trainer.model.push_to_hub(new_model)